In [11]:
# Dependencies and Setup
import pandas as pd
import requests
import json
import csv
import os
import gmaps
import pprint

#Load Google Places API key
from config import gkey

gmaps.configure(api_key=gkey)

# File to Load
cities = "cities.csv"

# Read city data file and store in Pandas Data frames
city_data = pd.read_csv(cities)
city_data.head()

,City,Country,Latitude,Longitude,Weather,Temperature,Humidity,Wind Speed,Cloud Cover %
0,Sambava,MG,-14.27,50.17,Rain,75.90,84,7.85,63
1,Chui,UY,-33.70,-53.46,Clear,64.87,75,7.85,0
2,Tasiilaq,GL,65.61,-37.64,Clouds,23.00,67,3.33,99
3,Yellowknife,CA,62.46,-114.35,Clouds,17.37,61,18.34,75
4,Yangi Marg`ilon,UZ,40.43,71.72,Clouds,50.00,61,2.24,100


In [12]:
# Extract the cities with ideal weather
ideal_temp = city_data[(city_data["Temperature"] >= 60) & (city_data["Temperature"] <= 80)]
no_wind = ideal_temp[ideal_temp["Wind Speed"] < 10]
clear_skys = no_wind[no_wind["Cloud Cover %"] < 10]
len(clear_skys)

39

In [13]:
response_json = []
name = []
vicinity = []
coords = []


for index, row in clear_skys.iterrows():
    lat = row["Latitude"]
    lng = row["Longitude"]
    

    # geocoordinates
    target_coordinates = f"{lat},{lng}"
    #print(lat)
    #print(lng)
    target_search = "Hotel"
    target_radius = 5000
    target_type = "hotel"

    # set up a parameters dictionary
    params = {
        "location": target_coordinates,
        "keyword": target_search,
        "radius": target_radius,
        "type": target_type,
        "key": gkey
    }

    # base url
    base_url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json"

    # run a request using our params dictionary
    response_json = requests.get(base_url, params=params).json()
    
    try:
        coords.append(target_coordinates)
        name.append(response_json["results"][0]["name"])
        vicinity.append(response_json["results"][0]["vicinity"])
        
    #response_json.append(response)
    
    except KeyError: 
        print("Cannot find city")
    except IndexError:
        print("No results")
        name.append("No results")
        vicinity.append("No results")

No results
No results
No results
No results
No results
No results


In [14]:
len(name)

39

In [15]:
# convert response to json
#response = response.json()

# Print the json (pretty printed)
print(json.dumps(response_json, indent=4, sort_keys=True))

{
    "html_attributions": [],
    "results": [],
    "status": "ZERO_RESULTS"
}


In [16]:
# convert response to json
#places_data = response.json()

# augment the google api data to the clear_skys dataframe

clear_skys["Hotel Vicinity"] = vicinity
clear_skys["Hotel Name"] = name

clear_skys

C:\ProgramData\Anaconda3\lib\site-packages\ipykernel_launcher.py:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  
C:\ProgramData\Anaconda3\lib\site-packages\ipykernel_launcher.py:7: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  import sys


,City,Country,Latitude,Longitude,Weather,Temperature,Humidity,Wind Speed,Cloud Cover %,Hotel Vicinity,Hotel Name
1,Chui,UY,-33.70,-53.46,Clear,64.87,75,7.85,0,"Rodovia BR-471 s/n - Centro, Chuí",Bertelli Chuí Hotel
16,Caborca,MX,30.58,-111.10,Clear,75.00,28,5.99,0,"Blvrd Guillermo Padrés 201, La Loma, Santa Ana",Motel Elba
27,Mar del Plata,AR,-38.00,-57.56,Clear,60.89,72,8.05,0,"Leandro N. Alem 4221, Mar del Plata",Sheraton Mar del Plata Hotel
28,Mercedes,AR,-34.65,-59.43,Clear,64.06,60,7.00,0,"Calle 16, Mercedes",Gran Hotel Mercedes
49,Dakar,SN,14.69,-17.44,Clear,75.65,78,6.93,0,"Route de la Corniche O, Dakar","Radisson Blu Hotel, Dakar Sea Plaza"
57,Diego de Almagro,CL,-26.37,-70.05,Clear,76.91,23,9.42,0,"Juan Martínez de Rosa 818, Atacama, Diego de A...",hotel luna del desierto
68,Cidreira,BR,-30.18,-50.21,Rain,66.56,73,9.04,3,"Estrada RS 784 KM 7 - Lagoa, Cidreira",Pond Country Club
97,Port Hedland,AU,-20.32,118.57,Clear,71.60,88,6.93,0,"Cnr Lukis And, McGregor St, Port Hedland",ibis Styles Port Hedland
103,Bragança Paulista,BR,-22.94,-46.56,Clear,66.90,72,5.82,0,"R. Renato de Oliveira, 80 - Jardim Sao Jose, B...",KA Business Hotel
112,Kumluca,TR,36.37,30.29,Clear,60.71,52,6.67,0,"Antalya, Kilise Yakasi Mevkii No 134, Kumluca",Simurg Evleri Olympos


In [17]:
locations = city_data[["Latitude", "Longitude"]]
humidity = city_data["Humidity"]

fig = gmaps.figure()

heat_layer = gmaps.heatmap_layer(locations, weights=humidity, dissipating=False, 
                                 max_intensity=10, point_radius=1)

fig.add_layer(heat_layer)

fig


Figure(layout=FigureLayout(height='420px'))

In [18]:
locations = clear_skys[["Latitude", "Longitude"]]
humidity = clear_skys["Humidity"]

fig = gmaps.figure()

heat_layer = gmaps.heatmap_layer(locations, weights=humidity, dissipating=False, 
                                 max_intensity=10, point_radius=1)

fig.add_layer(heat_layer)

fig

Figure(layout=FigureLayout(height='420px'))

In [19]:
# NOTE: Do not change any of the code in this cell

# Using the template add the hotel marks to the heatmap
info_box_template = """
<dl>
<dt>Name</dt><dd>{Hotel Name}</dd>
<dt>City</dt><dd>{City}</dd>
<dt>Country</dt><dd>{Country}</dd>
</dl>
"""
# Store the DataFrame Row
# NOTE: be sure to update with your DataFrame name
hotel_info = [info_box_template.format(**row) for index, row in clear_skys.iterrows()]
locations = clear_skys[["Latitude", "Longitude"]]

marker_layer = gmaps.marker_layer(locations, info_box_content=hotel_info)
fig.add_layer(marker_layer)
fig

Figure(layout=FigureLayout(height='420px'))